In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, PredefinedSplit
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor
import pandas as pd
import pickle

In [36]:
#Retrieve the data
x_train = pd.read_pickle("data/train_x.pkl")
x_val = pd.read_pickle("data/val_x.pkl")
x_test = pd.read_pickle("data/test_x.pkl")
x_nulls = pd.read_pickle("data/x_nulls.pkl")

y_train = pd.read_pickle("data/train_y.pkl")
y_val = pd.read_pickle("data/val_y.pkl")
y_test = pd.read_pickle("data/test_y.pkl")
y_nulls = pd.read_pickle("data/y_nulls.pkl")

n_train = len(x_train)
n_val = len(x_val)
n_test = len(x_test)
print(n_train, n_val, n_test)

5588 1197 1198


In [37]:
#Define prediction metrics
def forest_performance(model, x_train, y_train, x_val, y_val):
    prediction_train = model.predict(x_train)
    mae_train = mean_absolute_error(y_train , prediction_train)
    prediction_val = model.predict(x_val)
    mae_val = mean_absolute_error(y_val , prediction_val)
    print('Random forest train MAE = ', mae_train)
    print('Random forest validation MAE = ', mae_val)


In [38]:
#Run the model
model = RandomForestRegressor()
model.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [39]:
#There is a lot of overfitting
forest_performance(model, x_train, y_train, x_val, y_val)

Random forest train MAE =  7.61915767463652
Random forest validation MAE =  18.87397615661097


In [40]:
#Eliminate unimportant variables
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(x_train.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: distinct_products    Importance: 0.35
Variable: picking_speed        Importance: 0.12
Variable: distance             Importance: 0.08
Variable: found_rate           Importance: 0.06
Variable: quantity             Importance: 0.05
Variable: distinct_ratio       Importance: 0.04
Variable: rating               Importance: 0.03
Variable: quantity_ratio       Importance: 0.03
Variable: on_demand            Importance: 0.02
Variable: accepted_rate        Importance: 0.02
Variable: kg                   Importance: 0.02
Variable: kg_ratio             Importance: 0.02
Variable: store_branch_3871bd64012152bfb53fdf04b401193f Importance: 0.02
Variable: store_branch_1679091c5a880faf6fb5e6087eb1b2dc Importance: 0.01
Variable: store_branch_1afa34a7f984eeabdbb0a7d494132ee5 Importance: 0.01
Variable: store_branch_1f0e3dad99908345f7439f8ffabdffc4 Importance: 0.01
Variable: store_branch_45c48cce2e2d7fbdea1afc51c7c6ad26 Importance: 0.01
Variable: store_branch_52edc4a5890adc59cec82cb60f8af691 Imp

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [41]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
num_important = np.where(cumulative_importances >= 0.9)[0][0] + 1
print(num_important)
print(sorted_features[:19])
new_features = sorted_features[:19]

17
['distinct_products', 'picking_speed', 'distance', 'found_rate', 'quantity', 'distinct_ratio', 'rating', 'quantity_ratio', 'on_demand', 'accepted_rate', 'kg', 'kg_ratio', 'store_branch_3871bd64012152bfb53fdf04b401193f', 'store_branch_1679091c5a880faf6fb5e6087eb1b2dc', 'store_branch_1afa34a7f984eeabdbb0a7d494132ee5', 'store_branch_1f0e3dad99908345f7439f8ffabdffc4', 'store_branch_45c48cce2e2d7fbdea1afc51c7c6ad26', 'store_branch_52edc4a5890adc59cec82cb60f8af691', 'store_branch_5751ec3e9a4feab575962e78e006250d']


In [42]:
#Fit the model with the important variables only
x2_train = x_train[new_features]
x2_val = x_val[new_features]
model2 = RandomForestRegressor()
model2.fit(x2_train,y_train)
# n_estimators=100,max_features=0.3,max_depth=12,min_samples_leaf=3

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [43]:
#The model got worst on train set and on validation set, so lets better use hyperparameter tuning
forest_performance(model2, x2_train, y_train, x2_val, y_val)

Random forest train MAE =  7.939923755362962
Random forest validation MAE =  19.8094285328265


In [44]:
#Define a grid for the most important parameters to tune
min_samples_split = [2,6,10]
n_estimators =[100, 200, 300, 400]
max_features = [0.2, 0.5, 1]
max_depth = [5, 10, 20, 30, 40, 50]
min_samples_leaf= [1, 3, 5]

#Define the grid
random_grid = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,
               'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf}

In [45]:
# Fit the random search model
model_3 = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = model_3, param_distributions = random_grid, n_iter = 100, 
                               cv = 5, verbose=2, random_state= 30, n_jobs = -1, scoring="neg_mean_absolute_error")

rf_random.fit(x_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   57.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 23.3min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [100, 200, 300, 400], 'max_features': [0.2, 0.5, 1], 'max_depth': [5, 10, 20, 30, 40, 50], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 5]},
          pre_dispatch='2*n_jobs', random_state=30, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=2)

In [48]:
#Get the best parameters so far
print(rf_random.best_params_)

#Evaluate the new model
best_model = rf_random.best_estimator_
forest_performance(best_model, x_train, y_train, x_val, y_val)

{'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 0.2, 'max_depth': 40}
Random forest train MAE =  11.088603457982433
Random forest validation MAE =  17.778736991632012


In [49]:
#Refine the search in a grid search
#Define a grid for the most important parameters to tune
min_samples_split = [8, 10, 12]
n_estimators =[350, 400, 450]
max_features = [0.15, 0.2, 0.25]
max_depth = [35,40,45]
min_samples_leaf= [1]


#Define the grid
grid = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,
               'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf}

#Fit the model
model_4 = RandomForestRegressor()
rforest_grid = GridSearchCV(model_4, grid, cv = 3, verbose=2, n_jobs = -1, scoring="neg_mean_absolute_error")
rforest_grid.fit(x_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 41.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [350, 400, 450], 'max_features': [0.15, 0.2, 0.25], 'max_depth': [35, 40, 45], 'min_samples_split': [8, 10, 12], 'min_samples_leaf': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=2)

In [50]:
#Get the best parameters so far
print(rforest_grid.best_params_)

#Evaluate the new model
best_model_grid = rforest_grid.best_estimator_
forest_performance(best_model_grid, x_train, y_train, x_val, y_val)

{'max_depth': 45, 'max_features': 0.2, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 450}
Random forest train MAE =  10.319375998693053
Random forest validation MAE =  17.8148791715668


In [51]:
#Reported mean absolute error of 18.19
#The model doesnt work very well, as it tends to overfit with these amount of data
test_predictions = best_model_grid.predict(x_test)
mae_test = mean_absolute_error(y_test,test_predictions)
print('Mean absolute error in test set : ', mae_test)

Mean absolute error in test set :  18.196428411793995


In [52]:
#Dump the predictions
with open("predictions/random_forest.csv", "w") as f:
    predictions = best_model_grid.predict(x_nulls)
    [f.write("{}, {}\n".format(i,j)) for (i,j) in zip(y_nulls.order_id, predictions)]

In [55]:
#Dump the model
pickle.dump(best_model_grid, open("models_dump/random_forest.sav", 'wb'))